Exercise: distribution
-----------------

In this exercise we will create multiple Pods (instances) from the same Image (application) each with a ReplicaSet, Deployment and Service.

This happens in a separate namespace to be able to display the results in a targeted manner:

In [ ]:
! kubectl create namespace rs

Creating a deployment, here the example of Docker with a web server that outputs the current IP address.

In [ ]:
! # kubectl run apache --image=registry.gitlab.com/mc-b/misegr/hello-world --namespace rs",
! kubectl create deployment apache --image=registry.gitlab.com/mc-b/misegr/hello-world --namespace rs

Output of the YAML file that describes the generated resources.

The pod comes first after `spec.containers`.

In [ ]:
! kubectl get deployment apache -o yaml --namespace rs | kubectl-neat

To demonstrate the ReplicaSet functionality, let's set the number of running pods to 3 and look at the result

In [ ]:
! kubectl --namespace rs scale --replicas=3 deployment/apache  
# ! kubectl get pod,deployment,replicaset,service --namespace rs
! kube-lineage --namespace rs deployment apache -d 2 --exclude-types events 

For the **Deployment** `apache` we create a service. This makes the web server visible from the outside.

Port 80 is automatically mapped by Kubernetes to the next free port.

Let's look at the result again.

In [ ]:
! kubectl expose deployment/apache --type="LoadBalancer" --port 80 --namespace rs
# ! kubectl get pod,deployment,replicaset,service,endpoints -o wide --namespace rs
! kube-lineage --namespace rs deployment apache -d 2 --exclude-types events && echo ""
! kubectl get service -o wide --namespace rs

Since we don't have a load balancer, we have to prepare the IP of the cluster and the mapped port (port-based-routing) as a URL using a small shell script.

In [ ]:
! echo "http://"$(cat ~/work/server-ip)":"$(kubectl get service --namespace rs apache -o=jsonpath='{ .spec.ports[0].nodePort }')

If the link above is opened in multiple browsers, a different hostname should appear in each case.

**Explanation**: Kubernetes distributes the requests to the running pods.

- - -

To clean it up, just delete the namespace

In [ ]:
! kubectl delete namespace rs